# Resolução do Problema de Programação Linear - Jaisson Rações

---

- **Alunos:** Maruan Biasi El Achkar e Ricardo Falcão Schlieper
- **Professor:** Jaisson Potrich dos Reis
- **Matéria:** Pesquisa Operacional
- **Tema:** Otimização na produção de Rações utilizando Programação Linear - Trabalho N2
- **Link do Projeto:** https://github.com/JaissonGroup/JaissonRacoes

---

## Objetivo
- Maximizar o lucro total da safra, respeitando regras de agricultura, logística e produção de rações.

## Detalhamento do problema e regras de negócio:
- https://github.com/JaissonGroup/JaissonRacoes/tree/main/resolucao

## Website do Projeto
- https://jaissongroup.github.io/

---

In [2]:
# A ferramente PuLP será utilizada para resolver a programção linear
# https://pypi.org/project/PuLP/
!pip install PuLP

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 72.9 MB/s eta 0:00:00


In [3]:
from itertools import product
import pulp as pl

In [4]:
# CONJUNTOS

F = ["Maruim", "Gaucha", "CamposGerais", "Pantanal", "Cerrado", "Capital"]
G = ["Milho", "Soja", "Sorgo", "Trigo", "Aveia", "Girassol"]
L = ["BC-G", "BL-L", "AV-P", "EQ-M", "OV-M", "PA-PET", "AQ-T"]
Packs = ["std", "bb"]
I = ["Milho", "FareloSoja", "Sorgo", "FareloTrigo", "Aveia", "FareloGirassol", "Calcario", "Oleo", "Premix"]
K = ["pequeno", "medio", "grande"]

In [5]:
# embalagens permitidas por linha

P_of_L = {
    "BC-G": ["std", "bb"],
    "BL-L": ["std", "bb"],
    "AV-P": ["std", "bb"],
    "EQ-M": ["std"],       # sem bb
    "OV-M": ["std", "bb"],
    "PA-PET": ["std"],     # sem bb
    "AQ-T": ["std", "bb"],
}

In [6]:
# mapeamento grao -> ingrediente

G2I = {
    "Milho": "Milho",
    "Soja": "FareloSoja",
    "Sorgo": "Sorgo",
    "Trigo": "FareloTrigo",
    "Aveia": "Aveia",
    "Girassol": "FareloGirassol",
}

In [7]:
# PARÂMETROS (default=0)

# Agricultura
A = {}                 # area ha: A[f]
Padj = {}              # t/ha: Padj[f,g]
Csem = {}              # R$/ha: Csem[g]
Cman = {}              # R$/ha: Cman[g]

# Logística
D = {}                 # km (ida): D[f]
cap = {}               # t/viagem: cap[k]
ckm = {}               # R$/km: ckm[k]
cfix = {}              # R$/viagem: cfix[k]

# Rações
price = {}             # R$/item: price[l,p]
w = {}                 # kg/item: w[l,p]
cproc = {}             # R$/kg: cproc[l,p]
cpack = {}             # R$/item: cpack[l,p]
Qmax = {}              # itens: Qmax[l,p]

# Composição
alpha = {}             # fração: alpha[l,i] (somatório i = 1 por linha)

# Compra/Venda
buy = {}               # R$/t: buy[i]
sell = {}              # R$/t: sell[i]

# Políticas
gamma = 0.025          # min 2.5% por linha
hmin = 10.0            # ha mínimos se plantar


# helpers para buscar parâmetro com default=0 e chaves compostas
def par(d, key, default=0.0):
    return d.get(key, default)

def p2(l, p):
    return par

In [8]:
# PARÂMETROS — Jaisson Rações

# Agricultura (área ha)

A.update({
    "Maruim":        300.0,
    "Gaucha":        400.0,
    "CamposGerais":  500.0,
    "Pantanal":     1200.0,
    "Cerrado":      2500.0,
    "Capital":      2200.0,
})


# Produtividade ajustada Padj[f,g] (t/ha)

Padj.update({
    # Fazenda Maruim
    ("Maruim","Aveia"): 3.70,
    ("Maruim","Girassol"): 0.72,
    ("Maruim","Milho"): 4.20,
    ("Maruim","Soja"): 2.51,
    ("Maruim","Sorgo"): 1.02,
    ("Maruim","Trigo"): 3.96,
    # Fazenda Gaúcha
    ("Gaucha","Aveia"): 3.92,
    ("Gaucha","Girassol"): 0.00,
    ("Gaucha","Milho"): 6.00,
    ("Gaucha","Soja"): 3.57,
    ("Gaucha","Sorgo"): 0.00,
    ("Gaucha","Trigo"): 4.20,
    # Fazenda Campos Gerais
    ("CamposGerais","Aveia"): 5.60,
    ("CamposGerais","Girassol"): 0.72,
    ("CamposGerais","Milho"): 1.56,
    ("CamposGerais","Soja"): 0.99,
    ("CamposGerais","Sorgo"): 1.02,
    ("CamposGerais","Trigo"): 6.00,
    # Fazenda Pantanal
    ("Pantanal","Aveia"): 0.00,
    ("Pantanal","Girassol"): 2.41,
    ("Pantanal","Milho"): 8.64,
    ("Pantanal","Soja"): 5.55,
    ("Pantanal","Sorgo"): 4.54,
    ("Pantanal","Trigo"): 0.00,
    # Fazenda Cerrado
    ("Cerrado","Aveia"): 1.01,
    ("Cerrado","Girassol"): 2.41,
    ("Cerrado","Milho"): 6.00,
    ("Cerrado","Soja"): 4.03,
    ("Cerrado","Sorgo"): 4.54,
    ("Cerrado","Trigo"): 1.08,
    # Fazenda Capital
    ("Capital","Aveia"): 1.01,
    ("Capital","Girassol"): 2.41,
    ("Capital","Milho"): 6.00,
    ("Capital","Soja"): 4.03,
    ("Capital","Sorgo"): 4.54,
    ("Capital","Trigo"): 1.08,
})


# Custos agrícolas por ha (R$/ha)

Csem.update({
    "Milho": 670.0, "Soja": 483.0, "Sorgo": 294.0,
    "Girassol": 312.0, "Trigo": 410.0, "Aveia": 272.0
})
Cman.update({
    "Milho": 1785.0, "Soja": 2433.0, "Sorgo": 1692.0,
    "Girassol": 2364.0, "Trigo": 3024.0, "Aveia": 2327.0
})


# Distâncias (km ida)

D.update({
    "Maruim": 70.0,
    "Gaucha": 566.0,
    "CamposGerais": 228.0,
    "Pantanal": 954.0,
    "Cerrado": 2143.0,
    "Capital": 1287.0,
})


# Frota (capacidade t/viagem, custo por km, fixo por viagem)

cap.update({"pequeno":20.0, "medio":35.0, "grande":60.0})
ckm.update({"pequeno":8.0, "medio":10.5, "grande":12.0})
cfix.update({"pequeno":128.0, "medio":1064.0, "grande":9024.0})


# Pesos (kg/item) por linha/embalagem (std e bb quando existir)

w.update({
    ("BC-G","std"):30.0, ("BC-G","bb"):1000.0,
    ("BL-L","std"):30.0, ("BL-L","bb"):1000.0,
    ("AV-P","std"):25.0, ("AV-P","bb"):1000.0,
    ("EQ-M","std"):25.0,                    # sem bb
    ("OV-M","std"):30.0, ("OV-M","bb"):1000.0,
    ("PA-PET","std"):15.0,                  # sem bb
    ("AQ-T","std"):25.0, ("AQ-T","bb"):1000.0,
})


# Preços de venda (R$/item)

price.update({
    ("BC-G","std"): 78.00, ("BC-G","bb"): 2200.00,
    ("BL-L","std"): 88.00, ("BL-L","bb"): 2480.00,
    ("AV-P","std"): 75.00, ("AV-P","bb"): 2150.00,
    ("EQ-M","std"):105.00,                  # sem bb
    ("OV-M","std"): 82.00, ("OV-M","bb"): 2300.00,
    ("PA-PET","std"): 98.00,                # sem bb
    ("AQ-T","std"): 80.00, ("AQ-T","bb"): 2250.00,
})


# Custos de processo (R$/kg) e embalagem (R$/item ou R$/bb)

cproc.update({
    ("BC-G","std"):0.10, ("BC-G","bb"):0.10,
    ("BL-L","std"):0.11, ("BL-L","bb"):0.11,
    ("AV-P","std"):0.12, ("AV-P","bb"):0.12,
    ("EQ-M","std"):0.12,
    ("OV-M","std"):0.11, ("OV-M","bb"):0.11,
    ("PA-PET","std"):0.16,
    ("AQ-T","std"):0.24, ("AQ-T","bb"):0.24,
})
cpack.update({
    ("BC-G","std"): 5.00, ("BC-G","bb"): 95.00,
    ("BL-L","std"): 5.00, ("BL-L","bb"): 95.00,
    ("AV-P","std"): 4.50, ("AV-P","bb"): 95.00,
    ("EQ-M","std"): 4.50,
    ("OV-M","std"): 5.00, ("OV-M","bb"): 95.00,
    ("PA-PET","std"): 4.00,
    ("AQ-T","std"): 4.50, ("AQ-T","bb"): 95.00,
})


# Demanda máxima (itens)

Qmax.update({
    ("BC-G","std"):150000, ("BC-G","bb"):3000,
    ("BL-L","std"):110000, ("BL-L","bb"):2200,
    ("AV-P","std"):105000, ("AV-P","bb"):1125,
    ("EQ-M","std"): 60000,                   # sem bb
    ("OV-M","std"): 60000, ("OV-M","bb"):1200,
    ("PA-PET","std"):50000,                  # sem bb
    ("AQ-T","std"): 60000, ("AQ-T","bb"):1500,
})


# Composição (fração em massa por linha)
# I = ["Milho","FareloSoja","Sorgo","FareloTrigo","Aveia","FareloGirassol","Calcario","Oleo","Premix"]

alpha.update({
    # BC-G (Corte)
    ("BC-G","Milho"):0.55, ("BC-G","FareloSoja"):0.18, ("BC-G","Sorgo"):0.10,
    ("BC-G","FareloTrigo"):0.10, ("BC-G","Aveia"):0.00, ("BC-G","FareloGirassol"):0.03,
    ("BC-G","Calcario"):0.01, ("BC-G","Oleo"):0.02, ("BC-G","Premix"):0.01,
    # BL-L (Leite)
    ("BL-L","Milho"):0.345, ("BL-L","FareloSoja"):0.28, ("BL-L","Sorgo"):0.095,
    ("BL-L","FareloTrigo"):0.15, ("BL-L","Aveia"):0.05, ("BL-L","FareloGirassol"):0.05,
    ("BL-L","Calcario"):0.015, ("BL-L","Oleo"):0.005, ("BL-L","Premix"):0.01,
    # AV-P (Poedeira)
    ("AV-P","Milho"):0.55, ("AV-P","FareloSoja"):0.20, ("AV-P","Sorgo"):0.05,
    ("AV-P","FareloTrigo"):0.05, ("AV-P","Aveia"):0.02, ("AV-P","FareloGirassol"):0.00,
    ("AV-P","Calcario"):0.10, ("AV-P","Oleo"):0.02, ("AV-P","Premix"):0.01,
    # EQ-M (Equinos)
    ("EQ-M","Milho"):0.25, ("EQ-M","FareloSoja"):0.08, ("EQ-M","Sorgo"):0.00,
    ("EQ-M","FareloTrigo"):0.20, ("EQ-M","Aveia"):0.40, ("EQ-M","FareloGirassol"):0.03,
    ("EQ-M","Calcario"):0.01, ("EQ-M","Oleo"):0.02, ("EQ-M","Premix"):0.01,
    # OV-M (Ov/Capr)
    ("OV-M","Milho"):0.40, ("OV-M","FareloSoja"):0.20, ("OV-M","Sorgo"):0.10,
    ("OV-M","FareloTrigo"):0.15, ("OV-M","Aveia"):0.00, ("OV-M","FareloGirassol"):0.10,
    ("OV-M","Calcario"):0.02, ("OV-M","Oleo"):0.02, ("OV-M","Premix"):0.01,
    # PA-PET (Pássaros)
    ("PA-PET","Milho"):0.45, ("PA-PET","FareloSoja"):0.04, ("PA-PET","Sorgo"):0.00,
    ("PA-PET","FareloTrigo"):0.08, ("PA-PET","Aveia"):0.25, ("PA-PET","FareloGirassol"):0.15,
    ("PA-PET","Calcario"):0.01, ("PA-PET","Oleo"):0.01, ("PA-PET","Premix"):0.01,
    # AQ-T (Tilápia)
    ("AQ-T","Milho"):0.25, ("AQ-T","FareloSoja"):0.35, ("AQ-T","Sorgo"):0.10,
    ("AQ-T","FareloTrigo"):0.10, ("AQ-T","Aveia"):0.025, ("AQ-T","FareloGirassol"):0.10,
    ("AQ-T","Calcario"):0.005, ("AQ-T","Oleo"):0.05, ("AQ-T","Premix"):0.02,
})


# Compra/Venda (R$/t) para ingredientes mapeados aos grãos
# Tabelas do enunciado (grãos na fábrica, sem frete)

sell.update({
    "Milho": 450.0, "FareloSoja": 840.0, "Sorgo": 680.0,
    "FareloTrigo": 1260.0, "Aveia": 1020.0, "FareloGirassol": 1640.0,
    # os comprados "puros" normalmente não se vendem (deixe 0 se quiser)
    "Calcario": 0.0, "Oleo": 0.0, "Premix": 0.0,
})
buy.update({
    "Milho": 1380.0, "FareloSoja": 1380.0, "Sorgo": 1120.0,
    "FareloTrigo": 2060.0, "Aveia": 1670.0, "FareloGirassol": 2680.0,
    # preços dos ingredientes comprados direto (tabela dedicada)
    "Calcario": 400.0, "Oleo": 6000.0, "Premix": 15000.0,
})


In [9]:
# CONSTRUÇÃO DO MODELO

def build_model():
    m = pl.LpProblem("Jaisson_Racoes_MILP", pl.LpMaximize)


    # Variáveis

    # Agricultura
    x = pl.LpVariable.dicts("x", (F, G), lowBound=0.0, cat="Continuous")               # ha
    z = pl.LpVariable.dicts("z", (F, G), lowBound=0, upBound=1, cat="Binary")          # plantou?

    # Logística
    y = pl.LpVariable.dicts("y", (F, G, K), lowBound=0, cat="Integer")                 # viagens
    ship = pl.LpVariable.dicts("ship", (F, G), lowBound=0.0, cat="Continuous")         # t
    prod = pl.LpVariable.dicts("prod", (F, G), lowBound=0.0, cat="Continuous")         # t

    # Produção de ração
    q = { (l,p): pl.LpVariable(f"q_{l}_{p}", lowBound=0, cat="Integer")
          for l in L for p in P_of_L[l] }
    mmin = pl.LpVariable("m_min_25perc", lowBound=0, cat="Integer")                    # teto(2.5% do total)

    # Ingredientes
    buyQty  = pl.LpVariable.dicts("buyQty",  I, lowBound=0.0, cat="Continuous")        # t
    sellQty = pl.LpVariable.dicts("sellQty", I, lowBound=0.0, cat="Continuous")        # t
    use     = pl.LpVariable.dicts("use",     I, lowBound=0.0, cat="Continuous")        # t

    # Total de itens (auxiliar)
    Qtot = pl.lpSum(q[l, p] for l in L for p in P_of_L[l])


    # Função Objetivo

    # Receita rações
    revenue_feeds = pl.lpSum(par(price, (l,p)) * q[l,p] for l in L for p in P_of_L[l])

    # Venda de excedentes
    revenue_excess = pl.lpSum(par(sell, i) * sellQty[i] for i in I)

    # Custo agrícola por ha
    cost_agri = pl.lpSum( (par(Csem, g) + par(Cman, g)) * x[f][g] for f in F for g in G )

    # Frete (ida-e-volta): (2*D[f]*ckm[k] + cfix[k]) * y[f,g,k]
    cost_freight = pl.lpSum( (2*par(D, f)*par(ckm, k) + par(cfix, k)) * y[f][g][k]
                              for f in F for g in G for k in K )

    # Compras de ingredientes
    cost_buy = pl.lpSum(par(buy, i) * buyQty[i] for i in I)

    # Processo + embalagem (R$/kg * kg/item + R$/item) * itens
    cost_process_pack = pl.lpSum( (par(cproc, (l,p)) * par(w, (l,p)) + par(cpack, (l,p))) * q[l,p]
                                  for l in L for p in P_of_L[l] )

    m += (revenue_feeds + revenue_excess
          - cost_agri - cost_freight - cost_buy - cost_process_pack), "Lucro_Total"


    # Restrições

    # (R1) Uso total da área por fazenda
    for f in F:
        m += pl.lpSum(x[f][g] for g in G) == par(A, f), f"Area_total_{f}"

    # (R2) Plantio mínimo (10 ha se plantado)
    for f, g in product(F, G):
        m += x[f][g] >= hmin * z[f][g], f"Min_ha_if_planted_lb_{f}_{g}"
        m += x[f][g] <= par(A, f) * z[f][g], f"Min_ha_if_planted_ub_{f}_{g}"

    # (R3) Produção e embarque
    for f, g in product(F, G):
        m += prod[f][g] == par(Padj, (f,g)) * x[f][g], f"Prod_{f}_{g}"
        m += ship[f][g] == prod[f][g],                 f"ShipEq_{f}_{g}"

    # (R4) Capacidade de transporte (viagens inteiras)
    for f, g in product(F, G):
        m += pl.lpSum(par(cap, k) * y[f][g][k] for k in K) >= ship[f][g], f"Cap_{f}_{g}"

    # (R5) Balanço de ingredientes
    # sum_f ship[f,g] (mapeado para i) + buyQty[i] - sellQty[i] = use[i]
    for i in I:
        ships_i = []
        for f, g in product(F, G):
            if G2I.get(g) == i:
                ships_i.append(ship[f][g])
        m += pl.lpSum(ships_i) + buyQty[i] - sellQty[i] == use[i], f"Bal_{i}"

    # (R6) Consumo na produção: use[i] = sum_{l,p} alpha[l,i]*(w/1000)*q
    for i in I:
        m += use[i] == pl.lpSum( par(alpha, (l,i)) * (par(w, (l,p))/1000.0) * q[l,p]
                                 for l in L for p in P_of_L[l] ), f"Use_{i}"

    # (R7) Demanda máxima por linha/embalagem
    for l in L:
        for p in P_of_L[l]:
            m += q[l,p] <= par(Qmax, (l,p)), f"DemandaMax_{l}_{p}"

    # (R8) Mínimo 2,5% por linha (arredondado p/ cima)
    # mmin >= 0.025 * Qtot
    m += mmin >= gamma * Qtot, "MinShareCeil_def"
    # soma por linha >= mmin
    for l in L:
        m += pl.lpSum(q[l,p] for p in P_of_L[l]) >= mmin, f"MinShare_line_{l}"

    # (R 9) Presença de todos os tamanhos existentes
    for l in L:
        for p in P_of_L[l]:
            m += q[l,p] >= 1, f"Presence_{l}_{p}"

    return m, dict(
        x=x, z=z, y=y, ship=ship, prod=prod, q=q, mmin=mmin,
        buyQty=buyQty, sellQty=sellQty, use=use
    )

In [10]:

if __name__ == "__main__":
    model, vars_ = build_model()

    # Só estrutura - não resolve ainda.
    print("Modelo estruturado com sucesso.")
    print(f"Nº variáveis: {len(model.variables())}, Nº restrições: {len(model.constraints)}")

    # para ver o .lp
    model.writeLP("jaisson_milp.lp")


Modelo estruturado com sucesso.
Nº variáveis: 292, Nº restrições: 236


# RESOLVER

In [11]:
# RESOLUÇÃO COM ALGORITIMO CBC

if __name__ == "__main__":

    model, V = build_model()
    print("Modelo estruturado com sucesso.")
    print(f"→ Variáveis: {len(model.variables())} | Restrições: {len(model.constraints)}")

    # --- Resolve com CBC ---
    print("\n🔄 Resolvendo com CBC...")
    result_status = model.solve(pl.PULP_CBC_CMD(msg=True))
    print(f"\n✅ Status: {pl.LpStatus[result_status]}")
    print(f"💰 Lucro total: R$ {pl.value(model.objective):,.2f}")



    # RELATÓRIO

    x, z, y, ship, prod, q, mmin, buyQty, sellQty, use = (
        V["x"], V["z"], V["y"], V["ship"], V["prod"], V["q"],
        V["mmin"], V["buyQty"], V["sellQty"], V["use"]
    )

    print("\n🌾 ÁREA PLANTADA (ha) POR FAZENDA E GRÃO")
    for f in F:
        print(f"\nFazenda {f}:")
        for g in G:
            val = x[f][g].value()
            if val > 0.01:
                print(f"  - {g:<10}: {val:8.2f} ha")

    print("\n🚜 PRODUÇÃO (t) POR FAZENDA E GRÃO")
    for f in F:
        print(f"\nFazenda {f}:")
        for g in G:
            val = prod[f][g].value()
            if val > 0.1:
                print(f"  - {g:<10}: {val:10.2f} t")

    print("\n📦 PRODUÇÃO DE RAÇÕES (itens)")
    for l in L:
        total_line = sum(q[l,p].value() for p in P_of_L[l])
        if total_line > 0.1:
            print(f"\nLinha {l} — Total {int(total_line)} itens")
            for p in P_of_L[l]:
                val = q[l,p].value()
                if val > 0.1:
                    print(f"  {p:>4}: {int(val)}")

    print("\n🛒 COMPRAS DE INGREDIENTES (t)")
    for i in I:
        val = buyQty[i].value()
        if val > 0.01:
            print(f"  - {i:<15}: {val:10.2f} t")

    print("\n🏭 USO DE INGREDIENTES NA PRODUÇÃO (t)")
    for i in I:
        val = use[i].value()
        if val > 0.01:
            print(f"  - {i:<15}: {val:10.2f} t")

    print("\n💸 VENDA DE EXCEDENTES (t)")
    for i in I:
        val = sellQty[i].value()
        if val > 0.01:
            print(f"  - {i:<15}: {val:10.2f} t")

    print("\n🔢 Resumo final:")
    print(f"  - Lucro total (R$): {pl.value(model.objective):,.2f}")
    print(f"  - Mínimo (ceil 2.5%): {mmin.value():.0f} itens/linha")
    print(f"  - Status solver: {pl.LpStatus[result_status]}")


Modelo estruturado com sucesso.
→ Variáveis: 292 | Restrições: 236

🔄 Resolvendo com CBC...

✅ Status: Optimal
💰 Lucro total: R$ 30,189,579.03

🌾 ÁREA PLANTADA (ha) POR FAZENDA E GRÃO

Fazenda Maruim:
  - Aveia     :   300.00 ha

Fazenda Gaucha:
  - Milho     :   400.00 ha

Fazenda CamposGerais:
  - Trigo     :   481.67 ha
  - Aveia     :    18.33 ha

Fazenda Pantanal:
  - Milho     :   951.39 ha
  - Soja      :   248.61 ha

Fazenda Cerrado:
  - Sorgo     :    13.22 ha
  - Girassol  :  2486.78 ha

Fazenda Capital:
  - Milho     :    20.00 ha
  - Soja      :   997.52 ha
  - Sorgo     :   436.12 ha
  - Girassol  :   746.36 ha

🚜 PRODUÇÃO (t) POR FAZENDA E GRÃO

Fazenda Maruim:
  - Aveia     :    1110.00 t

Fazenda Gaucha:
  - Milho     :    2400.00 t

Fazenda CamposGerais:
  - Trigo     :    2890.00 t
  - Aveia     :     102.67 t

Fazenda Pantanal:
  - Milho     :    8220.00 t
  - Soja      :    1379.79 t

Fazenda Cerrado:
  - Sorgo     :      60.00 t
  - Girassol  :    5993.15 t

Fazend

In [12]:
# relatorio conforme enunciado do problema

def gerar_relatorio_completo(model, V):
    print("\n" + "="*70)
    print("📘 RELATÓRIO COMPLETO — JAISSON RAÇÕES")
    print("="*70)

    x, y, q, buyQty, sellQty = V["x"], V["y"], V["q"], V["buyQty"], V["sellQty"]


    # 1️⃣ Grãos plantados por fazenda
    print("\n🌾 GRÃOS PLANTADOS POR FAZENDA (ha)")
    custo_agri_fazenda = {}
    for f in F:
        total = 0
        print(f"\nFazenda {f}:")
        for g in G:
            ha = x[f][g].value()
            if ha > 0.01:
                custo = (Csem[g] + Cman[g]) * ha
                total += custo
                print(f"  - {g:<10}: {ha:8.2f} ha  |  Custo: R$ {custo:,.2f}")
        custo_agri_fazenda[f] = total
        print(f"  ➜ Custo agrícola total {f}: R$ {total:,.2f}")

    custo_agri_total = sum(custo_agri_fazenda.values())
    print(f"\n💰 CUSTO TOTAL AGRÍCOLA: R$ {custo_agri_total:,.2f}")


    # 2️⃣ Logística (viagens e custos)
    print("\n🚛 LOGÍSTICA — TRANSPORTES POR FAZENDA E CUSTOS")
    custo_log_fazenda = {}
    for f in F:
        total_f = 0
        print(f"\nFazenda {f}:")
        for g in G:
            for k in K:
                viagens = y[f][g][k].value()
                if viagens > 0.01:
                    custo = (2 * D[f] * ckm[k] + cfix[k]) * viagens
                    total_f += custo
                    print(f"  - {g:<10} | Caminhão {k:<7}: {viagens:6.0f} viagens | Custo: R$ {custo:,.2f}")
        custo_log_fazenda[f] = total_f
        print(f"  ➜ Custo logístico total {f}: R$ {total_f:,.2f}")

    custo_log_total = sum(custo_log_fazenda.values())
    print(f"\n💰 CUSTO TOTAL LOGÍSTICO: R$ {custo_log_total:,.2f}")


    # 3️⃣ Quantidade de sacos de ração
    print("\n📦 PRODUÇÃO DE RAÇÕES — ITENS E TAMANHOS")
    for l in L:
        print(f"\nLinha {l}:")
        for p in P_of_L[l]:
            val = q[l, p].value()
            if val > 0.01:
                print(f"  - {p:>3}: {int(val):10d} itens")


    # 4️⃣ Custo total com ingredientes comprados
    custo_ing = sum(buy[i] * buyQty[i].value() for i in I)
    print(f"\n🧾 CUSTO TOTAL COM INGREDIENTES COMPRADOS: R$ {custo_ing:,.2f}")


    # 5️⃣ Custo total de produção por linha e embalagem
    print("\n🏭 CUSTO DE PRODUÇÃO DE RAÇÕES (por linha e tamanho)")
    custo_prod_linha = {}
    for l in L:
        total_l = 0
        for p in P_of_L[l]:
            qnt = q[l, p].value()
            if qnt > 0.01:
                custo_unit = cproc[l, p] * w[l, p] + cpack[l, p]
                custo_total = custo_unit * qnt
                total_l += custo_total
                print(f"  - {l:<6} {p:<3}: R$ {custo_total:,.2f}")
        custo_prod_linha[l] = total_l

    custo_prod_total = sum(custo_prod_linha.values())
    print(f"\n💰 CUSTO TOTAL DE PRODUÇÃO DE RAÇÕES: R$ {custo_prod_total:,.2f}")


    # 6️⃣ Custos totais da empresa
    custo_oper_total = custo_agri_total + custo_log_total + custo_ing + custo_prod_total
    print(f"\n🧮 CUSTO TOTAL DE OPERAÇÃO: R$ {custo_oper_total:,.2f}")


    # 7️⃣ Faturamento total (rações + vendas de excedentes)
    faturamento_racoes = sum(price[l, p] * q[l, p].value() for l in L for p in P_of_L[l])
    faturamento_excedentes = sum(sell[i] * sellQty[i].value() for i in I)
    faturamento_total = faturamento_racoes + faturamento_excedentes
    print(f"\n💵 FATURAMENTO TOTAL: R$ {faturamento_total:,.2f}")


    # 8️⃣ Lucro final
    lucro_total = pl.value(model.objective)
    print(f"\n🏆 LUCRO FINAL (ótimo global): R$ {lucro_total:,.2f}")

    print("\n✅ RELATÓRIO COMPLETO GERADO COM SUCESSO\n")


if __name__ == "__main__":
    model, V = build_model()
    print("Modelo estruturado, resolvendo...\n")
    model.solve(pl.PULP_CBC_CMD(msg=False))
    gerar_relatorio_completo(model, V)


Modelo estruturado, resolvendo...


📘 RELATÓRIO COMPLETO — JAISSON RAÇÕES

🌾 GRÃOS PLANTADOS POR FAZENDA (ha)

Fazenda Maruim:
  - Aveia     :   300.00 ha  |  Custo: R$ 779,700.00
  ➜ Custo agrícola total Maruim: R$ 779,700.00

Fazenda Gaucha:
  - Milho     :   400.00 ha  |  Custo: R$ 982,000.00
  ➜ Custo agrícola total Gaucha: R$ 982,000.00

Fazenda CamposGerais:
  - Trigo     :   481.67 ha  |  Custo: R$ 1,654,043.34
  - Aveia     :    18.33 ha  |  Custo: R$ 47,648.33
  ➜ Custo agrícola total CamposGerais: R$ 1,701,691.68

Fazenda Pantanal:
  - Milho     :   951.39 ha  |  Custo: R$ 2,335,659.72
  - Soja      :   248.61 ha  |  Custo: R$ 724,950.00
  ➜ Custo agrícola total Pantanal: R$ 3,060,609.72

Fazenda Cerrado:
  - Sorgo     :    13.22 ha  |  Custo: R$ 26,246.70
  - Girassol  :  2486.78 ha  |  Custo: R$ 6,654,634.25
  ➜ Custo agrícola total Cerrado: R$ 6,680,880.95

Fazenda Capital:
  - Milho     :    20.00 ha  |  Custo: R$ 49,100.00
  - Soja      :   997.52 ha  |  Custo: R$ 2,908,